In [18]:
# k-nearest neighbor

# Import packages
import pandas as pd
import numpy as np
from collections import defaultdict
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import GridSearchCV


In [21]:
# importing from CSV file
training_data = pd.read_csv("../siren_data_train.csv", sep=",")
# print(f"Data size: {training_data.memory_usage().sum() / 1e6:.2f} MB")
training_data['heard'] = training_data['heard'].replace({'hearing':1, 'not_hearing':0})
# print(training_data['heard'])

# Create test set
X = training_data.drop(['heard'], axis=1)
Y = training_data['heard']
# print(X,Y)

# Splitting into 75% training and 25% test set
X_train, X_test, Y_train, Y_train = train_test_split(X, Y, test_size = 0.25, random_state = 1477, stratify = Y)


In [ ]:
# K-NN


